In [17]:
import pandas as pd

In [18]:
# Importation des données - on garde uniquement les colonnes et les dates qui nous intéressent

chunk_size = 1000000
file_path = "tr_mutualfunds S12.csv"
columns_to_keep = ['fdate', 'fundname', 'cusip', 'prc']

chunk_iterator = pd.read_csv(file_path, chunksize=chunk_size, usecols=columns_to_keep, dtype={'cusip': str})

processed_chunks = []

for i, chunk in enumerate(chunk_iterator):
    chunk_filtered = chunk[columns_to_keep]
    chunk_filtered['fdate'] = pd.to_datetime(chunk_filtered['fdate'], errors='coerce')
    chunk_filtered = chunk_filtered[(chunk_filtered['fdate'] >= '1974-01-01') & (chunk_filtered['fdate'] <= '2007-12-31')]

    processed_chunks.append(chunk_filtered)

data = pd.concat(processed_chunks)

In [23]:
result = data.groupby(['fundname', 'fdate'])['prc'].mean().reset_index()

rdt = []

for fonds in result['fundname'].unique():
    fonds_data = result[result['fundname'] == fonds]

    fonds_data['rdt'] = fonds_data['prc'].pct_change() * 100
    
    rdt.append(fonds_data)

tot_rdt = pd.concat(rdt)

In [ ]:
final_df = tot_rdt.merge(result[['fdate', 'fundname', 'prc']], on=['fdate', 'fundname'], how='left')

In [ ]:
output_file_path = 'mutual_funds.csv'
final_df.to_csv(output_file_path, index=False)